# Домашнее задание №6
## Проверка стат. гипотез. Исследование влияния различных параметров выборки на _p-value_
### PROC TTEST

_[Укажите в данной ячейке ФИО всех участников вашей группы - от 1 до 3 человек]_

Пучкин Данила Андреевич

### Задание

Чтобы лучше понять проблемы при использовании p-value, а также познакомиться с интересным примером ошибочных выводов, сделанных лишь на основе маленьких значений p-value, рекомендую прочитать [эту статью](https://www.nature.com/polopoly_fs/1.14700!/menu/main/topColumns/topLeftColumn/pdf/506150a.pdf). Кстати, не забудьте про _домашнее чтение_ - ссылка на слайдах.

В этом задании мы будем исследовать p-value для гипотезы о равенстве среднего выборки заданному числу: _**H0 = m**_. Для этого мы будем использовать процедуру `proc ttest;` в ее простейшем варианте.

Нужно провести симмуляцию выборок с различными характеристиками (т.е. сгенерировать выборки - выборки на каждой итерации симмуляции должны быть независимые) и проанализировать поведение p-value в различных ситуациях:
1. Как меняется p-value в зависимости от размера выборки **при соблюдении** предположений теста.
2. Как меняется p-value в зависимости от размера выборки **при нарушении** предположений теста.
3. Как ведет себя p-value в условиях **истинности H0**.  
4. Как ведет себя p-value когда **H0 не верна**.
5. Как ведет себя p-value **при соблюдении** предположений теста, когда **H0 не верна** в зависимости от размера выборки, когда выборки зависимые, т.е. увеличивая размер на 1, мы получаем еще одно наблюдение из той же последовательности сл. чисел. 

Игото: 5 симмуляции.

Результатом каждой симмуляции должен быть график _p-value в зависимости от кол-ва наблюдений_. Для каждой ситуации своя картика. Нужно (по возможности кратко) описать поведение p-value на картинках, соответствующих разным ситуациям.  


### 1. Генерация случайных чисел

В задании нужно провести симмуляцию. Здесь подразумевается, что мы просто используем генератор случайных чисел для того, чтобы создать случайную выборку заданного размера из заданного распределения.

Подробнее про генерацияю случайных чисел **нужно** прочитать [здесь](http://documentation.sas.com/?docsetId=lefunctionsref&docsetTarget=p026ygl6toz3tgn14lt4iu6cl5bb.htm&docsetVersion=9.4&locale=en) 

Нужно понимать:
- как использовать процедуру `call streaminit()` в data step для инициализации генератора случайных чисел.
- и ее параметр генерации seed. За что отвечает данный парметр? Как добиться построения одинаковых случайных последовательностей? Как добиться разных случайных последовательностей? Что такое воспроизводимое исследование?
- функция `rand()` в data step: ее параметры. 


### 2. Макрос для симмуляции

Нужно написать макрос, у которого есть параметры (_дополните список при необходимости_):
1. max_obs= - максимальное кол-во наблюдений в исследовании
2. dist= - распределение, из которого мы будем генерировать случайные числа. Два возможных значения: 'normal' и 'lognormal'.
3. mu= - среднее распределения в параметре dist=. 

Макрос должен:
1. Создать выборки (наборы данных) с кол-вом наблюдений от 2 до max_obs с единственной переменной value - случайные числа из распределения в параметре dist= и средним в параметре mu=.
2. На каждой построенной выборке запустить процедуру ttest для проверки гипотезы H0=0. 
3. Для каждой выборки сохранить соответсвующие кол-во степеней свободы - DF ( т.е. размер выборки - 1) и полученное значение p-value (все это выводится в результатах работы proc ttest) в набор данных. Этот набор будет содержать столько наблюдений, сколько выборок мы сгенерируем. 
4. По полученному набору построить требуемый график.



#### 2.1. Управление выводом процедуры, сохранение вывода в набор данных
Т.к. proc ttest при каждом запуске выводит минимум три таблицы (кстати, для любой процедуры SAS вы можете посмотреть на список выводимых таблиц в разделе документации "ODS Table Names". Вот [для ttest](http://go.documentation.sas.com/?docsetId=statug&docsetTarget=statug_ttest_details21.htm&docsetVersion=14.2&locale=en)),то с учетом того, что нам придется построить, например, 500 выборок, вывод процедуры (HTML5 в SAS Studio) будет очень большим. Нужно как-то его сохратить или отключить полностью. [В статье](https://blogs.sas.com/content/iml/2015/05/26/suppress-ods.html) рассказывается про различные методы управления выводом. Нам подойдет вариант с полным отключением вывода перед процедурой `ods exclude all;` с сохранением лишь нужной таблицы `ods output TTests=TTests ;`.

Также для ускорения работы можно отключить построение графиков в `proc ttest` - опция `plot=`.

#### 2.2. Объединение наборов данных
Все наборы, которые получатся при каждом запуске ttest, нужно объединить в один -  тут поможет `proc append`. Обратите внимание, что перед запуском нужно удалить общий набор - иначе наблюдения от разных симмуляций будут вместе - нам это не подходит.

#### 2.3. Построение графиков
Для построения графиков можно использовать proc sgplot например так:

`
proc sgplot data=alltests /* если здесь данные по всем тестам */;
    series x=DF y=probt;
    refline 0.1 0.05 0.01 / axis=y lineattrs=(color=red);
    reg x=DF  y=probt / lineattrs=(color=white);
run;
`

Посмотреть на возможности процедуры можно [тут](https://support.sas.com/sassamples/graphgallery/PROC_SGPLOT.html).


In [16]:
/* Здесь определение макроса для симмуляции */
%macro main(max_obs=, dist=, mu=, ind=);
    ods graphics off;
    ods exclude all;
    
    %if &ind EQ 1 %then %do;
        data _NULL_;
            call streaminit(0);
        run;
    %end;
    
    data alltests; 
        format DF 10. Probt 10.8
        stop;
    run;
    
    %do i = 2 %to &max_obs;
        data RandMT;
            %if &ind EQ 1 %then %do;
                call streaminit(rand('uniform'));
            %end; %else %do;
                call streaminit(0);
            %end;
            do i = 1 to &i;
                %if &dist EQ 'normal' %then %do;
                    value = rand('normal', &mu);
                %end; %else %do;
                    value = rand('lognormal');
                %end;
                output;
            end;
            drop i;
        run;
    
        
        proc ttest plots(unpack)=none
            data=RandMT
            H0=0;
            ods output TTests=OutTTests;
        run;
        
          
        proc append force 
            base=alltests 
            data=OutTTests; 
        run;
    %end;
    ods exclude none;
    ods graphics on;

    proc sgplot 
        data=alltests;
        series x=DF y=Probt; 
        refline 0.1 0.05 0.01 / axis=y lineattrs=(color=red); 
        reg x=DF y=Probt / lineattrs=(color=white); 
    run;

%mend main;


In [19]:
/* запуск симмуляции для случая 1: опишите, что это за случай  */
/* соблюдаются предположения о независимости и нормальности распределения выборок, гипотеза истинна */
%main(max_obs=2000, dist='normal', mu=0, ind=1);

#### _Ваш_ вывод по результатам симмуляции 1
...

P-value в среднем практически не изменяется при увеличении размера выборки. Можно сказать, что p-value рапределено расвномерно. Можно объяснить тем, что случайная выборка при соблюдении предположений может быть как очень совместимой со статистической моделью (нулевая гипотеза не отвргается), так и недостаточно совместимой.

In [20]:
/* запуск симмуляции для случая 2: опишите, что это за случай  */
/* не соблюдается предположение о нормальности распределения выборок*/
%main(max_obs=500, dist='lognormal', mu=0, ind=1);

#### _Ваш_ вывод по результатам симмуляции 2
...

P-value быстро становится малым, что объясняется несоблюдением предположения о нормальности распределения выборок, несмотря на их независимость. Гипотезу следует верно отвергнуть

In [21]:
/* запуск симмуляции для случая 3: опишите, что это за случай  */
/* соблюдаются предположения о независимости и нормальности распределения выборок, гипотеза ложна */
%main(max_obs=500, dist='normal', mu=3, ind=1);

#### _Ваш_ вывод по результатам симмуляции 3
...

P-value быстро становится малым, что объясняется ложностью гипотезы. Гипотезу следует верно отвергнуть

In [27]:
/* запуск симмуляции для случая 4: опишите, что это за случай  */
/* не соблюдаются предположения о независимости и нормальности распределения выборок*/
%main(max_obs=500, dist='lognormal', mu=0, ind=0);

#### _Ваш_ вывод по результатам симмуляции 4
...

P-value быстро становится малым, что, как и раньше, объясняется несоблюдением предположения о нормальности распределения выборок, на что не влияет зависимость выборок. Гипотезу следует верно отвергнуть

In [25]:
/* запуск симмуляции для случая 5: опишите, что это за случай  */
/* не соблюдается предположение о независимости выборок, гипотеза ложна */
%main(max_obs=500, dist='normal', mu=3, ind=0);

#### _Ваш_ вывод по результатам симмуляции 5
...

P-value быстро становится малым. Зависимость выборок не улучшает результат при ложной гипотезе. Ее следует верно отвергнуть